<a href="https://colab.research.google.com/github/viyaleta/DES/blob/master/DES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# TODO:
# add logging capability
# add stats capability

In [0]:
class Entity:
    def __init__(self, name, state='none', state_duration=0):
        self.name = name
        self.state = state
        self.state_duration = state_duration
    
            
class Resource(Entity):
    def __init__(self, name, in_use=False):
        Entity.__init__(self, name)
        self.in_use = in_use
        
        
class Recipient(Entity):
    def __init__(self, name, state='none', state_duration=0):
        Entity.__init__(self, name, state_duration=0)
        self.resource = None
        
        
class ServiceProvider(Entity):
    def __init__(self, name, available_states=[]):
        Entity.__init__(self, name)
        self.available_states = available_states
        self.user = None
    
    @property
    def available(self):
        return True if not self.user and self.state in self.available_states else False

In [0]:
### BASIC EVENTS ###

class Event:
    def __init__(self, obj, next_event=None):
        self.obj = obj
        self.next_event = next_event
        
    def process(self):
        return self.next_event, self.obj.state_duration
        
        
class ChangeState(Event):
    def __init__(self, obj, new_state='', new_state_duration=0, 
                 next_event=None):
        Event.__init__(self, obj, next_event)
        self.new_state = new_state
        self.new_state_duration = new_state_duration
        
    def process(self):
        old_state = self.obj.state
        self.obj.state = self.new_state
        self.obj.state_duration = self.new_state_duration
        print('{} changed state from {} to {}'
              .format(self.obj.name, old_state, self.new_state))
        
        return self.next_event, self.obj.state_duration
    
    
### EVENTS FOR RESOURCE ALLOCATION ###

class AllocateResource(Event):
    def __init__(self, obj, new_state='', new_state_duration=0, 
                 resources=[], next_event=None):
        Event.__init__(self, obj, next_event)
        self.resources = resources
        self.new_state = new_state
        self.new_state_duration = new_state_duration
        
    def process(self):
        resource_dibs = None
        
        # check if the resource is in use
        for resource in self.resources:
            if resource.in_use == False:
                resource_dibs = resource
                break
                
        # if nothing, wait one more tick
        if not resource_dibs:
            print('all resources are allocated... {} - {}'
                  .format(self.obj.name, self.obj.state))
            return self, 1
            
        # change state
        old_state = self.obj.state
        self.obj.state = self.new_state
        self.obj.state_duration = self.new_state_duration
        print('{} changed state from {} to {} - using {}'
              .format(self.obj.name, old_state, self.new_state, resource_dibs.name))
        
        # allocate the resource
        self.obj.resource = resource_dibs
        self.obj.resource.in_use = True
        
        return self.next_event, self.obj.state_duration
        
class DeallocateResource(Event):
    def __init__(self, obj, new_state='', new_state_duration=0, next_event=None):
        Event.__init__(self, obj, next_event)
        self.new_state = new_state
        self.new_state_duration = new_state_duration
        
    def process(self):
        # change state
        old_state = self.obj.state
        self.obj.state = self.new_state
        self.obj.state_duration = self.new_state_duration
        print('{} changed state from {} to {}'
              .format(self.obj.name, old_state, self.new_state))
        
        # change resource to not in use and remove from recipient
        self.obj.resource.in_use = False
        self.obj.resource = None
        
        return self.next_event, self.obj.state_duration
    
    
### EVENTS FOR SERVICE ALLOCATION ###


class GetService(Event):
    # links state of user to a service provider
    def __init__(self, obj, service_pool):
        Event.__init__(self, obj)
        self.service_pool = service_pool
        
    def process(self):
        service_dibs = None
        
        # allocate obj from pool to obj
        for service in self.service_pool:
            if service.available:
                service_dibs = service
                break
                
        if not service_dibs:
            return self, 1
        
        # link this object to service
        service_dibs.user = self.obj
        print('{} will now service {}'.format(service_dibs.name, service_dibs.user.name))
        return None, 0
    
    
class Service(Event):
    # services user using a service provider 
    def __init__(self, obj, new_state, new_state_duration, next_event=None):
        Event.__init__(self, obj, next_event)
        self.new_state = new_state
        self.new_state_duration = new_state_duration
        
    def process(self):        
        # waiting for a user
        if not self.obj.user:
            return self, 1
        
        self.obj.state = self.new_state
        self.obj.state_duration = self.new_state_duration
        self.obj.user.state = self.new_state
        self.obj.user.state_duration = self.new_state_duration
        
        print('{} and {} changed state to {}'.format(self.obj.name, self.obj.user.name, self.new_state))
        
        return self.next_event, self.obj.state_duration
    
    
class EndService(Event):
    # services user using a service provider and change state
    def __init__(self, obj, new_state, new_state_duration, next_event=None):
        Event.__init__(self, obj, next_event)
        self.new_state = new_state
        self.new_state_duration = new_state_duration
        
    def process(self):
        print('{} completed service for {}'.format(self.obj.name, self.obj.user.name))
        old_state = self.obj.state
        
        self.obj.user.state = 'none'
        self.obj.user.state_duration = 0
        print('{} changed state from {} to {}'.format(self.obj.user.name, old_state, self.obj.user.state))
        self.obj.user = None
        
        self.obj.state = self.new_state
        self.obj.state_duration = self.new_state_duration
        print('{} changed state from {} to {}'.format(self.obj.name, old_state, self.obj.state))

        return self.next_event, self.obj.state_duration
    

### CONVENIENCE FUNCTIONS ###

def link_events(events):
    for i in range(1, len(events)):
        events[i-1].next_event = events[i]
        
        
def circular_link_events(events):
    for i in range(1, len(events)):
        events[i-1].next_event = events[i]
    events[-1].next_event = events[0]

In [0]:
class Stream:
    def __init__(self, until=-1):
        self.time = 0
        self.future_events = []
        self.until = until
        
    def run(self):
        while True:
            
            # progress time
            self.progress_time()

            # for all events in the current time, process event
            while len(self.future_events)>0 and self.time==self.future_events[0][0]:
                _, event = self.future_events.pop(0)
                if event:
                    next_event, next_event_duration = event.process()

                    # schedule next event
                    self.schedule(next_event, next_event_duration)
           
            if self.break_condition():
                break
                
    def break_condition(self):
        if (self.time+1)==self.until:
            return True
        
        if len(self.future_events)==0:
            return True
        
        if self.future_events[0][0]>self.until:
            return True
                
    def progress_time(self):
        # grab time of the next event in the queue
        self.time = self.future_events[0][0]
        print('Time: {} '.format(self.time))
        
    def schedule(self, event, in_time):
        next_time = self.time + in_time
        
        # find the correct position to insert next event (proper time, last in queue)
        for i in range(len(self.future_events)+1):
            if i==len(self.future_events) or next_time < self.future_events[i][0]:
                self.future_events.insert(i, (next_time, event))
                break
                
                
class Wheel(Stream):
    def __init__(self, ticks):
        Stream.__init__(self)
        self.ticks = ticks
        self.wheel_turn = 0
        self.next_wheel_events = []
        
        
    def break_condition(self):
        if len(self.future_events)==0 and len(self.next_wheel_events)==0:
            return True
        
        
    def progress_time(self):            
        
        if len(self.future_events)>0 and self.time == self.future_events[0][0]:
            return  # don't progress - we still have more events to process
        
        # up time by one
        self.time += 1
        
        # if necessary, turn the wheel and replace the future events list
        if self.time == self.ticks:
            self.time = 0
            self.wheel_turn += 1
            self.future_events = self.next_wheel_events
            self.next_wheel_events = []
            
            # if there are events that have time > ticks, reschedule them
            for i in range(len(self.future_events)):
                if self.future_events[i][0]>=self.ticks:
                    event = self.future_events.pop(i)
                    self.schedule(event[1], 0)
        
        print('Wheel {} Time: {} '.format(self.wheel_turn, self.time))
        
    def schedule(self, event, in_time):
        next_time = self.time + in_time
        
        if next_time < self.ticks:
            # find the correct position to insert next event (proper time, last in queue)
            for i in range(len(self.future_events)+1):
                if i==len(self.future_events) or next_time < self.future_events[i][0]:
                    self.future_events.insert(i, (next_time, event))
                    break
                    
        # if next time doesn't fit in this wheel, place it into a future wheel
        else:
            next_time = next_time - self.ticks
            # find the correct position to insert next event (proper time, last in queue)
            for i in range(len(self.next_wheel_events)+1):
                if i==len(self.next_wheel_events) or next_time < self.next_wheel_events[i][0]:
                    self.next_wheel_events.insert(i, (next_time, event))
                    break

In [25]:
# simple car
car = Entity('car')

start = ChangeState(car, 'on', 1)
go = ChangeState(car, 'driving', 10)
stop = ChangeState(car, 'stopped', 1)

link_events([start, go, stop])

s = Stream()
s.schedule(start, 10)

s.run()

Time: 10 
car changed state from none to on
Time: 11 
car changed state from on to driving
Time: 21 
car changed state from driving to stopped
Time: 22 


In [26]:
# simple ATM machine

customer = Entity('customer')

queue = ChangeState(customer, 'in queue', 0)
use = ChangeState(customer, 'using machine', 3)
leave = ChangeState(customer, 'left', 0)
link_events([queue, use, leave])

s = Stream()
s.schedule(queue, 0)

s.run()

Time: 0 
customer changed state from none to in queue
customer changed state from in queue to using machine
Time: 3 
customer changed state from using machine to left


In [27]:
# simple bank simulation
customer = Recipient('customer')
ATM = Resource('ATM')

queue = ChangeState(customer, 'in queue', 0)
get_money = AllocateResource(customer, 'using machine', 3, [ATM])
leave = DeallocateResource(customer, 'left', 0)
link_events([queue, get_money, leave])

s = Stream()
s.schedule(queue, 0)

s.run()

Time: 0 
customer changed state from none to in queue
customer changed state from in queue to using machine - using ATM
Time: 3 
customer changed state from using machine to left


In [28]:
# queue bank simulation
customer_1 = Recipient('customer_1')
customer_2 = Recipient('customer_2')
ATM = Resource('ATM')

customer_1_arrives = ChangeState(customer_1, 'in queue', 0)
link_events([
    customer_1_arrives,
    AllocateResource(customer_1, 'using machine', 3, [ATM]),
    DeallocateResource(customer_1, 'left', 0)
])

customer_2_arrives = ChangeState(customer_2, 'in queue', 0)
link_events([
    customer_2_arrives,
    AllocateResource(customer_2, 'using machine', 3, [ATM]),
    DeallocateResource(customer_2, 'left', 0)
])

s = Stream()
s.schedule(customer_1_arrives, 0)
s.schedule(customer_2_arrives, 1)

s.run()

Time: 0 
customer_1 changed state from none to in queue
customer_1 changed state from in queue to using machine - using ATM
Time: 1 
customer_2 changed state from none to in queue
all resources are allocated... customer_2 - in queue
Time: 2 
all resources are allocated... customer_2 - in queue
Time: 3 
customer_1 changed state from using machine to left
customer_2 changed state from in queue to using machine - using ATM
Time: 6 
customer_2 changed state from using machine to left


In [29]:
# multiple resources bank simulation
customer_1 = Recipient('customer_1')
customer_2 = Recipient('customer_2')
customer_3 = Recipient('customer_3')
ATM_1 = Resource('ATM_1')
ATM_2 = Resource('ATM_2')

customer_1_arrives = ChangeState(customer_1, 'in queue', 0)
link_events([
    customer_1_arrives,
    AllocateResource(customer_1, 'using machine', 3, [ATM_1, ATM_2]),
    DeallocateResource(customer_1, 'left', 0)
])

customer_2_arrives = ChangeState(customer_2, 'in queue', 0)
link_events([
    customer_2_arrives,
    AllocateResource(customer_2, 'using machine', 3, [ATM_1, ATM_2]),
    DeallocateResource(customer_2, 'left', 0)
])

customer_3_arrives = ChangeState(customer_3, 'in queue', 0)
link_events([
    customer_3_arrives,
    AllocateResource(customer_3, 'using machine', 3, [ATM_1, ATM_2]),
    DeallocateResource(customer_3, 'left', 0)
])

s = Stream()
s.schedule(customer_1_arrives, 0)
s.schedule(customer_2_arrives, 1)
s.schedule(customer_3_arrives, 2)

s.run()

Time: 0 
customer_1 changed state from none to in queue
customer_1 changed state from in queue to using machine - using ATM_1
Time: 1 
customer_2 changed state from none to in queue
customer_2 changed state from in queue to using machine - using ATM_2
Time: 2 
customer_3 changed state from none to in queue
all resources are allocated... customer_3 - in queue
Time: 3 
customer_1 changed state from using machine to left
customer_3 changed state from in queue to using machine - using ATM_1
Time: 4 
customer_2 changed state from using machine to left
Time: 6 
customer_3 changed state from using machine to left


In [54]:
# simple car - timing wheel
car = Entity('car')

start = ChangeState(car, 'idle', 1)

link_events([
    start, 
    ChangeState(car, 'driving', 10),
    ChangeState(car, 'waiting at light', 2),
    ChangeState(car, 'driving', 14),
    ChangeState(car, 'stopped', 10)
])

w = Wheel(16)
w.schedule(start, 10)

w.run()

Wheel 0 Time: 1 
Wheel 0 Time: 2 
Wheel 0 Time: 3 
Wheel 0 Time: 4 
Wheel 0 Time: 5 
Wheel 0 Time: 6 
Wheel 0 Time: 7 
Wheel 0 Time: 8 
Wheel 0 Time: 9 
Wheel 0 Time: 10 
car changed state from none to idle
Wheel 0 Time: 11 
car changed state from idle to driving
Wheel 0 Time: 12 
Wheel 0 Time: 13 
Wheel 0 Time: 14 
Wheel 0 Time: 15 
Wheel 1 Time: 0 
Wheel 1 Time: 1 
Wheel 1 Time: 2 
Wheel 1 Time: 3 
Wheel 1 Time: 4 
Wheel 1 Time: 5 
car changed state from driving to waiting at light
Wheel 1 Time: 6 
Wheel 1 Time: 7 
car changed state from waiting at light to driving
Wheel 1 Time: 8 
Wheel 1 Time: 9 
Wheel 1 Time: 10 
Wheel 1 Time: 11 
Wheel 1 Time: 12 
Wheel 1 Time: 13 
Wheel 1 Time: 14 
Wheel 1 Time: 15 
Wheel 2 Time: 0 
Wheel 2 Time: 1 
Wheel 2 Time: 2 
Wheel 2 Time: 3 
Wheel 2 Time: 4 
Wheel 2 Time: 5 
car changed state from driving to stopped
Wheel 2 Time: 6 
Wheel 2 Time: 7 
Wheel 2 Time: 8 
Wheel 2 Time: 9 
Wheel 2 Time: 10 
Wheel 2 Time: 11 
Wheel 2 Time: 12 
Wheel 2 Time: 13 


In [42]:
# Airport Taxi Allocation (2 people)

person_1 = Entity('person_1')
person_2 = Entity('person_2')

taxi_1 = ServiceProvider('taxi_1', available_states=['waiting'])

person_1_intent = ChangeState(person_1, 'in queue', 0, GetService(person_1, [taxi_1]))
person_2_intent = ChangeState(person_2, 'in queue', 0, GetService(person_2, [taxi_1]))

taxi_1_waiting = ChangeState(taxi_1, 'waiting', 0)
circular_link_events([
    taxi_1_waiting,
    Service(taxi_1, 'traveling downtown', 20),
    EndService(taxi_1, 'traveling to airport', 18),
])

s = Stream(until=100)
s.schedule(person_1_intent, 1)
s.schedule(person_2_intent, 2)
s.schedule(taxi_1_waiting, 3)
s.run()

Time: 1 
person_1 changed state from none to in queue
Time: 2 
person_2 changed state from none to in queue
Time: 3 
taxi_1 changed state from none to waiting
>>> taxi_1 will now service person_1
taxi_1 and person_1 changed state to traveling downtown
Time: 4 
Time: 5 
Time: 6 
Time: 7 
Time: 8 
Time: 9 
Time: 10 
Time: 11 
Time: 12 
Time: 13 
Time: 14 
Time: 15 
Time: 16 
Time: 17 
Time: 18 
Time: 19 
Time: 20 
Time: 21 
Time: 22 
Time: 23 
taxi_1 completed service for person_1
person_1 changed state from traveling downtown to none
taxi_1 changed state from traveling downtown to traveling to airport
Time: 24 
Time: 25 
Time: 26 
Time: 27 
Time: 28 
Time: 29 
Time: 30 
Time: 31 
Time: 32 
Time: 33 
Time: 34 
Time: 35 
Time: 36 
Time: 37 
Time: 38 
Time: 39 
Time: 40 
Time: 41 
taxi_1 changed state from traveling to airport to waiting
>>> taxi_1 will now service person_2
taxi_1 and person_2 changed state to traveling downtown
Time: 61 
taxi_1 completed service for person_2
person_2 chan

In [45]:
# Airport Taxi Allocation BUG BUG BUG

person_1 = Entity('person_1')
person_2 = Entity('person_2')
person_3 = Entity('person_3')
person_4 = Entity('person_4')
person_5 = Entity('person_5')
taxi_1 = ServiceProvider('taxi_1', ['waiting'])
taxi_2 = ServiceProvider('taxi_2', ['waiting'])

person_1_intent = ChangeState(person_1, 'in queue', 0, GetService(person_1, [taxi_1, taxi_2]))
person_2_intent = ChangeState(person_2, 'in queue', 0, GetService(person_2, [taxi_1, taxi_2]))
person_3_intent = ChangeState(person_3, 'in queue', 0, GetService(person_3, [taxi_1, taxi_2]))
person_4_intent = ChangeState(person_4, 'in queue', 0, GetService(person_4, [taxi_1, taxi_2]))
person_5_intent = ChangeState(person_5, 'in queue', 0, GetService(person_5, [taxi_1, taxi_2]))


taxi_1_waiting = ChangeState(taxi_1, 'waiting', 0)
circular_link_events([
    taxi_1_waiting,
    Service(taxi_1, 'traveling downtown', 20),
    EndService(taxi_1, 'traveling to airport', 18),
])

taxi_2_waiting = ChangeState(taxi_2, 'waiting', 0)
circular_link_events([
    taxi_2_waiting,
    Service(taxi_2, 'traveling downtown', 15),
    EndService(taxi_2, 'traveling to airport', 14),
])

s = Stream(until=155)
s.schedule(person_1_intent, 1)
s.schedule(person_2_intent, 4)
s.schedule(person_3_intent, 5)
s.schedule(person_4_intent, 16)
s.schedule(person_5_intent, 10)
s.schedule(taxi_1_waiting, 0)
s.schedule(taxi_2_waiting, 5)

s.run()

Time: 0 
taxi_1 changed state from none to waiting
Time: 1 
person_1 changed state from none to in queue
>>> taxi_1 will now service person_1
Time: 2 
taxi_1 and person_1 changed state to traveling downtown
Time: 4 
person_2 changed state from none to in queue
Time: 5 
person_3 changed state from none to in queue
taxi_2 changed state from none to waiting
>>> taxi_2 will now service person_2
taxi_2 and person_2 changed state to traveling downtown
Time: 6 
Time: 7 
Time: 8 
Time: 9 
Time: 10 
person_5 changed state from none to in queue
Time: 11 
Time: 12 
Time: 13 
Time: 14 
Time: 15 
Time: 16 
person_4 changed state from none to in queue
Time: 17 
Time: 18 
Time: 19 
Time: 20 
taxi_2 completed service for person_2
person_2 changed state from traveling downtown to none
taxi_2 changed state from traveling downtown to traveling to airport
Time: 21 
Time: 22 
taxi_1 completed service for person_1
person_1 changed state from traveling downtown to none
taxi_1 changed state from traveling dow